In [2]:
#UPLOAD MY_FUNCTION, DOWNLOAD FROM MY GITHUB PAGE AND UPLOAD HERE niazwazir===>googlecolab====>myfunction.py
from google.colab import files
src = list(files.upload().values())[0]
open('mylib.py','wb').write(src)
import mylib

Saving myfun.py to myfun.py


In [22]:
#UPLOAD BABY IMAGE, DOWNLAOD FROM MY GITHUB PAGE AND UPLOAD HERE ====>niazwazir=====>tbaby_GT.bmp
from google.colab import files
src = list(files.upload().values())[0]
open('mylib.py','wb').write(src)
import mylib

Saving baby_GT.bmp to baby_GT.bmp


In [13]:
#UPLOAD TEST FILE, DOWNLAOD FROM MY GITHUB PAGE AND UPLOAD HERE ====>niazwazir=====>testscale3.h5
from google.colab import files
src = list(files.upload().values())[0]
open('mylib.py','wb').write(src)
import mylib

Saving testscale3.h5 to testscale3.h5


In [5]:
#UPLOAD TRAINING FILE, DOWNLAOD FROM MY GITHUB PAGE AND UPLOAD HERE ====>niazwazir=====>trainscale.h5
from google.colab import files
src = list(files.upload().values())[0]
open('mylib.py','wb').write(src)
import mylib

Saving trainscale3.h5 to trainscale3.h5


In [0]:
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, Activation, Input
import keras.backend as K
from keras import optimizers
import numpy as np
import h5py
import scipy.misc
from keras.models import load_model
import matplotlib.pyplot as plt
import myfun
import cv2
import numpy 
import math
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

In [0]:
img_shape = (32,32,1)
input_img = Input(shape=(img_shape))
C1 = Conv2D(64,(9,9),padding='SAME',name='CONV1')(input_img)
A1 = Activation('relu', name='act1')(C1)
C2 = Conv2D(32,(1,1),padding='SAME',name='CONV2')(A1)
A2 = Activation('relu', name='act2')(C2)
C3 = Conv2D(1,(5,5),padding='SAME',name='CONV3')(A2)
A3 = Activation('relu', name='act3')(C3)
model = Model(input_img, A3)

In [43]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 32, 32, 1)         0         
_________________________________________________________________
CONV1 (Conv2D)               (None, 32, 32, 64)        5248      
_________________________________________________________________
act1 (Activation)            (None, 32, 32, 64)        0         
_________________________________________________________________
CONV2 (Conv2D)               (None, 32, 32, 32)        2080      
_________________________________________________________________
act2 (Activation)            (None, 32, 32, 32)        0         
_________________________________________________________________
CONV3 (Conv2D)               (None, 32, 32, 1)         801       
_________________________________________________________________
act3 (Activation)            (None, 32, 32, 1)         0         
Total para

In [0]:
f = h5py.File('trainscale3.h5','r')
input_ = f.get('data')[:]
target_ = f.get('label')[:]
f.close()

f = h5py.File('testscale3.h5','r')
input_val = f.get('data')[:]
target_val = f.get('label')[:]
f.close()

In [70]:
opt = optimizers.Adam(lr=0.0003)
model.compile(optimizer=opt,loss='mean_squared_error')
model.fit(input_,target_,batch_size = 32, epochs = 1,validation_data=(input_val,target_val))
model.save('wazirmodelscale3A.h5')

Train on 17202 samples, validate on 1991 samples
Epoch 1/1
17202/17202 [==============================] - 7s 408us/step - loss: 260.4339 - val_loss: 145.4386


In [71]:
img_o = scipy.misc.imread('baby_GT.bmp',flatten=True, mode='YCbCr').astype(np.float)
img = myfun.create_LR(img_o,4)
img_size = 32
stride = 16
h,w = img.shape
piece_wise = []
for x in range(0, h-img_size+1, stride):
    for y in range(0, w-img_size+1, stride):
        sub_input = img[x:x+img_size, y:y+img_size].reshape(img_size,img_size,1) # [32 x 32]
        piece_wise.append(sub_input)
input_ = np.asarray(piece_wise)        
srcnn = load_model('wazirmodelscale3A.h5')
hat = srcnn.predict(input_)
img_re = np.zeros(img.shape)
i=0
for x in range(0, h-img_size+1, stride):
    for y in range(0, w-img_size+1, stride):
        img_re[x:x+img_size, y:y+img_size] = hat[i].reshape(img_size,img_size)
        i += 1
cv2.imwrite('restored1.bmp',img_re)
cv2.imwrite('HR1.bmp',img_o)
img_save = (img*255).astype(np.uint8)
cv2.imwrite('blurred1.bmp',img_save)

True

In [72]:
#CALCULATE PSNR
original = cv2.imread("HR1.bmp")
contrast = cv2.imread("blurred1.bmp",1)
def psnr(img1, img2):
    mse = numpy.mean( (img1 - img2) ** 2 )
    if mse == 0:
            return 100
    PIXEL_MAX = 255.0
    return 20 * math.log10(PIXEL_MAX / math.sqrt(mse))

d=psnr(original,contrast)
print(d)

32.483873839424604
